# TP 1: recommandations par filtres collaboratifs utilisateur-utilisateur et item-item 

In [1]:
### Si données locales
u.user <- read.csv(file='u.user.csv', sep='|', header=T)
u.item <- read.csv(file='u.item.csv', sep='|', header=T)
u.data <- read.csv(file='u.data.csv', sep='|', header=T)


library(Matrix)
m.sparse <- sparseMatrix(u.data[,1],u.data[,2],x=u.data[,3])
rownames(m.sparse) <- paste(1:nrow(m.sparse), sep='')
colnames(m.sparse) <- u.item$movie.title


Quelque fonctions utiles:

In [2]:
max.nindex <- function(m, n=5) {
  i <- order(m, decreasing=TRUE)
  return(i[1:n])
}
min.nindex <- function(m, n=5) {
  i <- order(m)
  return(i[1:n])
}

# Q1 
*Quelle est la moyenne des votes par profession ("job") et par âge?*

On va dans un premier temps effectuer un "merge" entre les utilisateurs et les votes, pour obtenir facilement le job et ages correspondant à chaque vote

In [3]:
user_data <- merge(u.data, u.user, by.x = 'user.id', by.y = 'id')

on va ensuite faire faire la moyenne de tout les votes, en groupant par jobs

In [4]:
# moyenne par profession
head(aggregate(user_data$rating, by=list(user_data$job), FUN=mean))

Group.1,x
<fct>,<dbl>
administrator,3.635646
artist,3.653380
doctor,3.688889
educator,3.670621
engineer,3.541407
entertainment,3.441050


On effectue la même chose pour l'age

In [5]:
# moyenne par age
head(aggregate(user_data$rating, by=list(user_data$age), FUN=mean))

Group.1,x
<int>,<dbl>
7,3.767442
10,3.387097
11,2.925926
13,3.470825
14,3.375000
15,3.264484


# Q2
*Quels sont les 10 films les plus similaires à "Star Trek V: The Final Frontier (1989)" selon respectivement la mesure du cosinus et de la corrélation avec la matrice de votes.*


### Avec le cosinus

Dans un premier temps, on selectionne le vecteur de vote correspondant a star trek

In [6]:
start_trek_vect <- m.sparse[,'Star Trek V: The Final Frontier (1989)']

Ensuite, on calcul le cosinus de ce vecteur et de tout les autres vecteurs des autres films

In [7]:
## Cosinus entre un vecteur v et chaque colonne dela matrice m
cosinus.vm <- function(v,m) {
    n <- sqrt(colSums(m^2)); (v %*% m)/(n * sqrt(sum(v^2))) 
}

mat_cos <- cosinus.vm(start_trek_vect, m.sparse)

Enfin, on selectionne seulement les 11 premiers films qui ont le plus grand cosinus (Star Trek V: The Final Frontier (1989) et les 10 autres films les plus similaires)

In [8]:
top_10_index_cos <- max.nindex(mat_cos, n = 11)
top_10_cos <- u.item$movie.title[top_10_index_cos]
show(top_10_cos)

 [1] Star Trek V: The Final Frontier (1989)       
 [2] Star Trek: The Motion Picture (1979)         
 [3] Star Trek VI: The Undiscovered Country (1991)
 [4] Star Trek III: The Search for Spock (1984)   
 [5] Star Trek IV: The Voyage Home (1986)         
 [6] Star Trek: The Wrath of Khan (1982)          
 [7] Stargate (1994)                              
 [8] Star Trek: Generations (1994)                
 [9] Die Hard 2 (1990)                            
[10] Escape from New York (1981)                  
[11] Conan the Barbarian (1981)                   
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)


### Avec la corrélation de Pearson

Pour la corrélation, une fonction native existe dans les packages par défaut de R
On utilise cette fonction sur chaque item de la matrice m.sparse

In [9]:
## corrélation de Pearson entre un vecteur v et chaque colonne dela matrice m

cor.vm <- function(v, m) {
    # ici, le parametre 2 indique que nous effectuons la corrélation
    # sur les colones et non sur les lignes
    apply(m, 2, cor, y=v, use="pairwise.complete.obs")
}
mat_cor = cor.vm(start_trek_vect, m.sparse)

On applique ensuite le même raisonnement que pour la mesure du cosinus

In [10]:
top_10_index_cor <- max.nindex(mat_cor, n = 11)
top_10_cor <- u.item$movie.title[top_10_index_cor]
show(top_10_cor)

 [1] Star Trek V: The Final Frontier (1989)       
 [2] Star Trek: The Motion Picture (1979)         
 [3] Star Trek VI: The Undiscovered Country (1991)
 [4] Star Trek III: The Search for Spock (1984)   
 [5] Star Trek IV: The Voyage Home (1986)         
 [6] Star Trek: The Wrath of Khan (1982)          
 [7] Stargate (1994)                              
 [8] Star Trek: Generations (1994)                
 [9] Die Hard 2 (1990)                            
[10] Escape from New York (1981)                  
[11] Conan the Barbarian (1981)                   
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)


## Q3 
*Utilisez une approche item-item pour calculer le vote au film "Star Trek V: The Final Frontier (1989)" des utilisateurs qui n'ont pas de vote pour celui-ci. Prenez les 20 voisins les plus rapprochés selon la distance euclidienne et utilisez les cosinus comme poids. Si aucun vote commun existe, alors la valeur prédite est fixée à NA.*

Dans un premier temps, il faut calculer les distances de chaque filme par rapport a star trek
On va utiliser la distance.450 définie dans l'aide fournie, qui conserve les valeures manquantes à 0 pour le calcul de la distance

In [11]:
distance.450 <- sqrt(colSums((m.sparse[,450] - m.sparse)^2)) # valeurs manquantes à 0

On va ensuite determiner les 20 voisins les plus proches, ce sont ceux dont la distance est minimal (sans prendre en compte star strek)

In [12]:
voisins_star_trek = min.nindex(distance.450, 21)[2:21]

In [13]:
u.item$movie.title[voisins_star_trek]

[1] Robocop 3 (1993)                                
 [2] Mr. Magoo (1997)                                
 [3] Vampire in Brooklyn (1995)                      
 [4] Jerky Boys, The (1994)                          
 [5] Across the Sea of Time (1995)                   
 [6] Blood For Dracula (Andy Warhol's Dracula) (1974)
 [7] Fair Game (1995)                                
 [8] Phat Beach (1996)                               
 [9] S.F.W. (1994)                                   
[10] Window to Paris (1994)                          
[11] Farmer & Chase (1995)                           
[12] Fire on the Mountain (1996)                     
[13] Open Season (1996)                              
[14] Tough and Deadly (1995)                         
[15] Daniel Defoe's Robinson Crusoe (1996)           
[16] Blood Beach (1981)                              
[17] Paris Was a Woman (1995)                        
[18] Bad Girls (1994)                                
[19] Mad Dog Time (1996)                             
[20] Shooter, The (1995)                             
1664 Levels: 'Til There Was You (1997) 1-900 (1994) ... Zeus and Roxanne (1997)

On applique ensuite la pondération par cosinus sur ces voisins pour déterminer les notes manquantes

In [14]:
poids_voisins <- cosinus.vm(start_trek_vect, m.sparse[, voisins_star_trek])
head(as.matrix(poids_voisins))

Robocop 3 (1993),Mr. Magoo (1997),Vampire in Brooklyn (1995),"Jerky Boys, The (1994)",Across the Sea of Time (1995),Blood For Dracula (Andy Warhol's Dracula) (1974),Fair Game (1995),Phat Beach (1996),S.F.W. (1994),Window to Paris (1994),Farmer & Chase (1995),Fire on the Mountain (1996),Open Season (1996),Tough and Deadly (1995),Daniel Defoe's Robinson Crusoe (1996),Blood Beach (1981),Paris Was a Woman (1995),Bad Girls (1994),Mad Dog Time (1996),"Shooter, The (1995)"
0.3180997,0.2790087,0.2580293,0.2318994,0.207867,0.2027901,0.2342691,0.1891935,0.1903723,0.1891935,0.1891935,0.1891935,0.1835447,0.1855195,0.1692198,0.1817712,0.1418951,0.1577983,0.1418951,0.1576916


On cherche maintenant a avoir la liste des utilisateurs qui ont voté pour au moins 1 des 20 films les plus proches, et qui n'ont pas voté pour star trek

In [15]:
# on selectionne les utilisateurs qui n'ont pas voté pour star trek
user_index_non_vote_start_treck <- which(start_trek_vect == 0)
# on croise avec les utilisateurs qui ont voté pour au moins 1 des films parmis les 20 les plus proches
user_to_process <- intersect(which(rowSums(m.sparse[, voisins_star_trek]) !=0), user_index_non_vote_start_treck)
show(user_to_process)

 [1]  21  37  82 109 110 181 345 358 404 425 537 577 609 615 617 634 642 671 729
[20] 749 756 805 811 813 833 854 863 893 932


On applique maintenant la formule vue en cours pour calculer les notes de chaque utilisateur conserné

- On retire les valeures 0 pour calculer les moyennes  

In [16]:
start_trek_vect[start_trek_vect == 0] <- NA
m_sparse_voisin <- m.sparse[, voisins_star_trek]
m_sparse_voisin[m_sparse_voisin == 0] <- NA

mean_note_voisins <- apply(m_sparse_voisin, 2, mean, na.rm = TRUE)
mean_note_star_treck <- mean(start_trek_vect, na.rm = TRUE)
head(as.matrix(mean_note_voisins), 100)
print(cat("moyenne pour star treck: ", mean_note_star_treck))

Robocop 3 (1993),1.727273
Mr. Magoo (1997),1.916667
Vampire in Brooklyn (1995),1.833333
"Jerky Boys, The (1994)",2.666667
Across the Sea of Time (1995),2.750000
Blood For Dracula (Andy Warhol's Dracula) (1974),2.400000
Fair Game (1995),2.181818
Phat Beach (1996),1.600000
S.F.W. (1994),3.000000
Window to Paris (1994),4.000000
Farmer & Chase (1995),4.000000


moyenne pour star treck:  2.396825NULL


- On créer une nouvelle matrice centrée sur la moyenne des films voisins (correspondant à la somme des termes $v_{i,j}-\bar{v_{i}}$). On fait cela pour ne pas prendre en compte les valeurs manquantes, misent à 0 dans m.sparse, et qui doivent rester à 0 pour ne pas compter dans le somme

In [17]:
m.sparse.to_center <- m.sparse[, voisins_star_trek]
m.sparse.to_center[m.sparse.to_center == 0] <- NA

m.sparse.centered <- m.sparse.to_center[user_to_process,] - mean_note_voisins
m.sparse.centered[is.na(m.sparse.centered)] <- 0
head(as.matrix(m.sparse[user_to_process, voisins_star_trek]))
head(as.matrix(m.sparse.centered))

,Robocop 3 (1993),Mr. Magoo (1997),Vampire in Brooklyn (1995),"Jerky Boys, The (1994)",Across the Sea of Time (1995),Blood For Dracula (Andy Warhol's Dracula) (1974),Fair Game (1995),Phat Beach (1996),S.F.W. (1994),Window to Paris (1994),Farmer & Chase (1995),Fire on the Mountain (1996),Open Season (1996),Tough and Deadly (1995),Daniel Defoe's Robinson Crusoe (1996),Blood Beach (1981),Paris Was a Woman (1995),Bad Girls (1994),Mad Dog Time (1996),"Shooter, The (1995)"
21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
82,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
109,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,2,0,0
181,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


,Robocop 3 (1993),Mr. Magoo (1997),Vampire in Brooklyn (1995),"Jerky Boys, The (1994)",Across the Sea of Time (1995),Blood For Dracula (Andy Warhol's Dracula) (1974),Fair Game (1995),Phat Beach (1996),S.F.W. (1994),Window to Paris (1994),Farmer & Chase (1995),Fire on the Mountain (1996),Open Season (1996),Tough and Deadly (1995),Daniel Defoe's Robinson Crusoe (1996),Blood Beach (1981),Paris Was a Woman (1995),Bad Girls (1994),Mad Dog Time (1996),"Shooter, The (1995)"
21,0,0,0.00000000,0,0,0,0,0.0,0,0,0,0,0,0,0,-1.166667,0,0.0000000,0,0.0
37,0,0,0.00000000,0,0,0,0,0.0,0,0,0,0,0,0,0,0.000000,0,0.0000000,0,0.5
82,0,0,0.00000000,0,0,0,0,-1.4,0,0,0,0,0,0,0,0.000000,0,0.0000000,0,0.0
109,0,0,0.08333333,0,0,0,0,0.0,0,0,0,0,0,0,0,0.000000,0,0.0000000,0,0.0
110,0,0,0.00000000,0,0,0,0,0.0,0,0,0,0,0,0,0,0.000000,0,0.1666667,0,0.0
181,0,0,0.00000000,0,0,0,0,-2.0,0,0,0,0,-2,0,0,0.000000,0,0.0000000,0,0.0


- On applique la formule

In [18]:
notes_prediction <- mean_note_star_treck + (m.sparse.centered %*% t(poids_voisins)) / sum(poids_voisins)
head(as.matrix(notes_prediction))

21,2.343788
37,2.416545
82,2.330582
109,2.402203
110,2.403403
181,2.210384


On peut voir que la différence est très minime entre les notes. Cela est due au fait que $sum(w_{a,i})$ est calculée en conservant les valeures manquante dans le compte.
Une autre solution plus intelligente serais de mettre a zero le coefficient $w_{a,i}$, là ou le vote $i,j$ n'est pas présent. Pour cela, nous allons conserver la solution précedente (via les moyenne) qui gère bien les valeures manquante au dénominateur, et nous concentrer uniquement sur le recalcul du dénomiateur. Pour cela, nous allons pondérer la somme via une matrice des votes binaire (0 si le vote n'a pas eu lieu, et 1 si le vote a eu lieu). On ne prendre en compte que les 20 films les plus proches et les utilisateurs qui ont voté sur au moins 1 de ces films:

In [19]:
m.sparse.binaire <- m.sparse[user_to_process, voisins_star_trek]
m.sparse.binaire[m.sparse.binaire != 0] <- 1
head(as.matrix(m.sparse.binaire))

,Robocop 3 (1993),Mr. Magoo (1997),Vampire in Brooklyn (1995),"Jerky Boys, The (1994)",Across the Sea of Time (1995),Blood For Dracula (Andy Warhol's Dracula) (1974),Fair Game (1995),Phat Beach (1996),S.F.W. (1994),Window to Paris (1994),Farmer & Chase (1995),Fire on the Mountain (1996),Open Season (1996),Tough and Deadly (1995),Daniel Defoe's Robinson Crusoe (1996),Blood Beach (1981),Paris Was a Woman (1995),Bad Girls (1994),Mad Dog Time (1996),"Shooter, The (1995)"
21,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
37,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
82,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
109,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
181,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0


Une fois cette matrice créée, nous allons effectuer le produit matricielle de la matrice avec des poids calculée auparavant, et effectuer la somme sur les films pour obtenir $w_{a,i}$ de chaque utilisateur n'ayant pas voté.

In [20]:
poids_voisins.pondere <- m.sparse.binaire %*% t(poids_voisins)
show(sum(poids_voisins))
head(as.matrix(poids_voisins.pondere))

[1] 3.998445


21,0.1817712
37,0.1576916
82,0.1891935
109,0.2580293
110,0.3920674
181,0.3727382


Nous n'avons plus besoin de normaliser la matrice des votes car lorsque $v_{i,j}$ sera manquante, la valeur $w_{a,i}$ sera nul et la valeur manquante ne sera pas prise en compte

In [21]:
notes_prediction <- mean_note_star_treck + (m.sparse.centered %*% t(poids_voisins)) / poids_voisins.pondere
head(as.matrix(notes_prediction), 100)

21,1.2301587
37,2.8968254
82,0.9968254
109,2.4801587
110,2.4639050
181,0.3968254
345,1.7301587
358,2.8968254
404,2.5634921
425,1.7301587
537,0.7301587


### Q4
*Calculez l'erreur quadratique moyenne des prédiction de l'approche item-item à la question précédente en la comparant aux valeurs observées.*

Pour calculer l'erreur quadratique, nous allons devoir trouver les utilisateurs qui ont voté pour le films, et qui ont eux aussi voté pour les 20 films les plus proches

In [22]:
# on selectionne les utilisateurs qui n'ont pas voté pour star trek
user_index_start_treck <- which(start_trek_vect != 0)
# on croise avec les utilisateurs qui ont voté pour au moins 1 des films parmis les 20 les plus proches
user_to_process <- intersect(which(rowSums(m.sparse[, voisins_star_trek]) !=0), user_index_start_treck)
show(user_to_process)

 [1]   7  13 127 130 145 267 276 279 303 394 399 405 417 497 532 600 650 774 807


On reconstruit notre matrice de vote centrée sur la moyenne des notes des films voisins

In [23]:
m.sparse.to_center <- m.sparse[, voisins_star_trek]
m.sparse.to_center[m.sparse.to_center == 0] <- NA

m.sparse.centered <- m.sparse.to_center[user_to_process,] - mean_note_voisins
m.sparse.centered[is.na(m.sparse.centered)] <- 0

On va ensuite prédire leurs notes:

In [24]:
# on re calcul la matrice des votes binaires
m.sparse.binaire <- m.sparse[user_to_process, voisins_star_trek]
m.sparse.binaire[m.sparse.binaire != 0] <- 1

# On calcul les poids pondérés par utilisateurs
poids_voisins.pondere <- m.sparse.binaire %*% t(poids_voisins)

# on prédit leurs notes
notes_prediction <- mean_note_star_treck + (m.sparse.centered %*% t(poids_voisins)) / poids_voisins.pondere
head(as.matrix(notes_prediction))

7,2.9193662
13,2.2732315
127,5.4801587
130,1.6522810
145,0.7301587
267,1.7301587


Nous allons maintenant calculer l'erreur quadratique vis a vis des notes réelles

In [25]:
erreur_quadra <- sqrt(sum((notes_prediction - m.sparse[user_to_process, 'Star Trek V: The Final Frontier (1989)'])**2)/length(notes_prediction))
print(cat("erreure quadratique: ", erreur_quadra))
head(as.matrix(notes_prediction), 100)
head(as.matrix(m.sparse[user_to_process, 'Star Trek V: The Final Frontier (1989)']), 100)

erreure quadratique:  1.089232NULL


7,2.9193662
13,2.2732315
127,5.4801587
130,1.6522810
145,0.7301587
267,1.7301587
276,1.4049780
279,3.5693350
303,1.7912787
394,4.2150072
399,0.2925599


7,4
13,3
127,5
130,2
145,3
267,2
276,1
279,4
303,3
394,3
399,2


### Q5
*Utilisez une approche par agglomération (clustering) pour prédire les votes et calculez l'erreur quadratique moyenne.*

Tout d'abord on construit une matrice qui ne contient pas de valeur manquantes à 0

Pour effectuer l'algoritme k-mean, il est préférable de remplacer les valeures manquantes par les moyennes par films ou par utilisateurs. Ici, comme nous voulons grouper les utilisateurs, nous allons corriger la matrice avec les moyennes respectives des films.

In [26]:
library(stats)

# on normalise avec les moyennes par utilisateurs
m.sparse.for_kmean_by_user <- t(apply(m.sparse.to_normalisee, 1, normalization_by_mean))
# on normalise avec les moyennes par films
m.sparse.for_kmean_by_film <- apply(m.sparse.to_normalisee, 2, normalization_by_mean)

Ensuite, on utilise la fonction native du module stats de R pour clusterisé les utilisateurs, et définir entre 5 et 20 utilisateurs "moyens"

In [27]:
library(stats)
k_mean_obj <- kmeans(m.sparse.for_kmean_by_film, 5)
centers5 <- as.matrix(k_mean_obj$centers)
clusters5 <- as.matrix(k_mean_obj$cluster)
head(centers5)

Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),⋯,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
3.878319,1.000000,3.033333,4.000000,4.000000,3.576923,3.798469,4.000000,3.896321,3.831461,⋯,3,4,3,2,3,1,3,2,3,3
3.858416,3.200185,3.028427,3.531087,3.307028,3.584647,3.765152,3.975505,3.900461,3.829529,⋯,3,4,3,2,3,1,3,2,3,3
3.270257,2.896708,2.854167,3.329022,3.095567,3.484976,3.543367,3.810003,3.630801,3.761587,⋯,3,4,3,2,3,1,3,2,3,3
4.122726,3.263253,3.096296,3.908426,3.282300,3.580662,4.199617,4.247971,4.266676,3.938358,⋯,3,4,3,2,3,1,3,2,3,3
4.331326,3.487100,3.166245,3.573557,3.435973,3.576923,3.946138,4.098838,3.735786,3.808420,⋯,3,4,3,2,3,1,3,2,3,3


In [28]:
k_mean_obj <- kmeans(m.sparse.for_kmean_by_film, 10)
centers10 <- as.matrix(k_mean_obj$centers)
clusters10 <- as.matrix(k_mean_obj$cluster)
head(centers10, 10)

Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),⋯,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
4.131127,3.212860,3.067949,3.797571,3.265951,3.600592,4.117380,4.151270,4.114827,3.940795,⋯,3,4,3,2,3,1,3,2,3,3
3.878319,1.000000,3.033333,4.000000,4.000000,3.576923,3.798469,4.000000,3.896321,3.831461,⋯,3,4,3,2,3,1,3,2,3,3
3.072061,2.535078,2.736508,3.178628,2.901440,3.521978,3.361759,3.474451,3.655041,3.689139,⋯,3,4,3,2,3,1,3,2,3,3
3.861154,3.197524,3.029959,3.521880,3.305589,3.586419,3.770141,3.981808,3.943536,3.832159,⋯,3,4,3,2,3,1,3,2,3,3
4.028035,3.254123,3.036896,3.548669,3.340316,3.576923,3.819053,3.996305,3.677653,3.818852,⋯,3,4,3,2,3,1,3,2,3,3
3.000000,3.206107,2.000000,3.550239,3.302326,1.000000,4.000000,3.995434,4.000000,2.000000,⋯,3,4,3,2,3,1,3,2,3,3
4.471579,3.633588,3.310256,3.598822,3.542636,3.576923,4.035191,4.126449,3.813309,3.767502,⋯,3,4,3,2,3,1,3,2,3,3
3.476083,3.158377,2.964211,3.455956,3.266340,3.557085,3.593153,3.913002,3.717127,3.826611,⋯,3,4,3,2,3,1,3,2,3,3
4.193329,3.389313,3.169231,4.307692,3.309481,3.455621,4.522959,4.691956,4.753280,3.870354,⋯,3,4,3,2,3,1,3,2,3,3
3.000000,3.000000,3.033333,5.000000,1.000000,3.576923,2.000000,4.000000,3.000000,3.831461,⋯,3,4,3,2,3,1,3,2,3,3


In [29]:
k_mean_obj <- kmeans(m.sparse.for_kmean_by_film, 20)
centers20 <- as.matrix(k_mean_obj$centers)
clusters20 <- as.matrix(k_mean_obj$cluster)
head(centers20, 20)

Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),⋯,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998),You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
3.000000,4.000000,3.033333,4.000000,3.302326,3.576923,5.000000,3.995434,3.896321,4.000000,⋯,3,4,3,2,3,1,3,2,3,3
3.000000,3.000000,3.033333,5.000000,1.000000,3.576923,2.000000,4.000000,3.000000,3.831461,⋯,3,4,3,2,3,1,3,2,3,3
3.942101,3.204642,3.056863,3.563527,3.317085,3.587045,3.802663,4.011493,3.923812,3.830000,⋯,3,4,3,2,3,1,3,2,3,3
4.428734,3.566967,3.090909,3.650138,3.398872,3.576923,3.786951,4.270790,3.715618,3.760981,⋯,3,4,3,2,3,1,3,2,3,3
5.000000,4.000000,3.033333,3.000000,3.302326,3.576923,3.798469,5.000000,3.896321,3.831461,⋯,3,4,3,2,3,1,3,2,3,3
3.595870,3.126136,2.879365,3.495329,3.168328,3.617216,4.009232,3.807567,3.945692,3.927234,⋯,3,4,3,2,3,1,3,2,3,3
2.904357,2.371110,2.710256,3.084652,3.055456,3.488166,3.153846,3.075869,3.644456,3.588591,⋯,3,4,3,2,3,1,3,2,3,3
4.168142,3.206107,2.918519,3.972355,3.157623,3.623932,3.754535,4.554033,4.065032,4.610487,⋯,3,4,3,2,3,1,3,2,3,3
3.878319,1.000000,3.033333,4.000000,4.000000,3.576923,3.798469,4.000000,3.896321,3.831461,⋯,3,4,3,2,3,1,3,2,3,3
4.094087,3.273976,3.136970,3.674728,3.440169,3.576923,4.053989,3.905521,3.786683,3.846782,⋯,3,4,3,2,3,1,3,2,3,3


On va maintenant essayer de prédire la note de Star treck en déterminant dans quel cluster l'utilisateur se trouve et quelle est la note attribuée par le cluster

In [30]:
get_prediction <- function(clusters, centers) {

    clusters <- as.data.frame(clusters)
    clusters$user_name = rownames(clusters)
    
    centers <- as.data.frame(centers)
    centers$cluster_num <- rownames(centers)

    # on fusionne les tables clusters et centers pour obtenir les prédictions pour chaque utilisateur en fonction de son cluster d'appartenance
    prediction <- merge.data.frame(clusters, centers, by.x = "V1", by.y = "cluster_num")

    # on retrie les prédictions par utilisateur pour pouvoir selectionner les utilisateurs par index
    rownames(prediction) <- prediction$user_name
    return(prediction[order(as.numeric(prediction$user_name)),])
}

prediction5 <- get_prediction(clusters5, centers5)
head(prediction5)
# on selectionne seulement les notes des utilisateurs ayant voté star treck
user_star_treck_prediction <- prediction5[user_index_start_treck,]

head(user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"])
head(m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)'])

V1,user_name,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),⋯,Mirage (1995),Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998).1,You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991)
<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,1,4.122726,3.263253,3.096296,3.908426,3.282300,3.580662,4.199617,4.247971,⋯,3,4,3,2,3,1,3,2,3,3
2,2,3.858416,3.200185,3.028427,3.531087,3.307028,3.584647,3.765152,3.975505,⋯,3,4,3,2,3,1,3,2,3,3
2,3,3.858416,3.200185,3.028427,3.531087,3.307028,3.584647,3.765152,3.975505,⋯,3,4,3,2,3,1,3,2,3,3
2,4,3.858416,3.200185,3.028427,3.531087,3.307028,3.584647,3.765152,3.975505,⋯,3,4,3,2,3,1,3,2,3,3
3,5,3.270257,2.896708,2.854167,3.329022,3.095567,3.484976,3.543367,3.810003,⋯,3,4,3,2,3,1,3,2,3,3
2,6,3.858416,3.200185,3.028427,3.531087,3.307028,3.584647,3.765152,3.975505,⋯,3,4,3,2,3,1,3,2,3,3


,Star Trek V: The Final Frontier (1989)
,<dbl>
5,2.281746
7,2.422178
13,2.281746
28,2.397240
38,2.480812
44,2.397240


5  7 13 28 38 44 
 1  4  3  1  1  2

In [31]:
# on calcule l'erreur quadratique entre les vrais notes des utilisateurs et la note de leurs clusters restpectif
sum_err <- sum(abs(user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)']))
err_abs_moy <- sum_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"])
sum_sqrt_err <- sum((user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)'])**2)
erreur_quadra <- sqrt(sum_sqrt_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"]))

print("erreur quadra/moyenne pour cluster 5")
show(erreur_quadra)
show(err_abs_moy)

[1] "erreur quadra/moyenne pour cluster 5"
[1] 1.130067
[1] 0.9534742


In [32]:
# on fait de même avec les clusters 10 et 20
prediction10 <- get_prediction(clusters10, centers10)
# on selectionne seulement les notes des utilisateurs ayant voté star treck
user_star_treck_prediction <- prediction10[user_index_start_treck,]

# on calcule l'erreur quadratique entre les vrais notes des utilisateurs et la note de leurs clusters restpectif

sum_err <- sum(abs(user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)']))
err_abs_moy <- sum_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"])
sum_sqrt_err <- sum((user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)'])**2)
erreur_quadra <- sqrt(sum_sqrt_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"]))

print("erreur quadra/moyenne pour cluster 10")
show(erreur_quadra)
show(err_abs_moy)

prediction20 <- get_prediction(clusters20, centers20)
# on selectionne seulement les notes des utilisateurs ayant voté star treck
user_star_treck_prediction <- prediction20[user_index_start_treck,]

# on calcule l'erreur quadratique entre les vrais notes des utilisateurs et la note de leurs clusters restpectif
sum_err <- sum(abs(user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)']))
err_abs_moy <- sum_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"])
sum_sqrt_err <- sum((user_star_treck_prediction["Star Trek V: The Final Frontier (1989)"] - m.sparse[user_index_start_treck, 'Star Trek V: The Final Frontier (1989)'])**2)
erreur_quadra <- sqrt(sum_sqrt_err / length(user_star_treck_prediction[,"Star Trek V: The Final Frontier (1989)"]))

print("erreur quadra/absolue pour cluster 20")
show(erreur_quadra)
show(err_abs_moy)

[1] "erreur quadra/moyenne pour cluster 10"
[1] 1.109414
[1] 0.9223064
[1] "erreur quadra/absolue pour cluster 20"
[1] 1.072882
[1] 0.8772294


### Q6
*Je suis un nouvel utilisateur. Vous connaissez ma profession, mon sexe et mon âge. Développez un algorithme bayésien pour recommander 10 films sur la base de ces trois catégories.*

Pour recommander 10 films sur base de ces trois catégories, nous avons décider d'estimer pour chaque film la valeur attendue du vote en calculant la somme pondérée de la probabilité de chaque valeur par sa probabilité :  $E(v) = \sum_{i=1}^5 P(v=i|profession,age,genre) \ i$ 

Comme vu au cours, en postulant l'indépendance des facteurs, on peut réécrire ces probabilités comme suit : $ P(v=i|profession,age,genre) = \frac{1}{Z} \ P(v=i) \ P(profession|v=i) \ P(age|v=i) \ P(genre|v=i)$. 

Étant donné que la valeur $Z$ ne dépend pas des facteurs et que nous ne cherchons pas à estimer exactement les votes mais seulement à trouver les 10 meilleurs films, nous pouvons ignorer cette valeur.

Pour calculer la $P(F|v=i)$ de chaque film (où $F$ est un des facteurs *profession*, *age* ou *genre*), nous utilisons la correction de Laplace. C'est-à-dire que nous calculons le ratio $\frac{\#(F,v=i) + 1}{\#(v=i) + n}$ 
où $\#(F,v=i)$ est le nombre votes du film de valeurs $i$ donné par des utilisateurs répondant au facteur $F$, $\#(v=i)$ est le nombre total de votes de valeur i pour ce film et $n$ est le nombre de professions/age/genres différents. 

In [ ]:
Bayes_recommendation <- function(job, age, gender){
    
  n_item <- ncol(m.sparse)

  data_user <- merge(u.data, u.user, by.x='user.id', by.y='id')
  
  # matrice 1682x5 contenant les probabailités P(v=i) pour i=1...5 et pour chaque film
  n_ratings_by_film <- with(data_user, table(item.id, rating, dnn = c("film.id", "rating")))
  
  # vecteur comprenant le nombre de votes total pour chaque film
  sum_ratings <- apply(n_ratings_by_film, 1, sum)
  
  # matrice 1682x5 où l'élément (i,j) correspond à la probabilité que le film i reçoive une note de j 
  prob_ratings_by_film <- (n_ratings_by_film + 1)/(sum_ratings + 5)
  
  index_job <- data_user[,'job'] == job
  
  index_age <- data_user[,'age'] == age
  
  index_gender <- data_user[,'gender'] == gender
  
  n_job <- length(table(data_user[,'job']))
  
  n_age <- length(table(data_user[,'age']))
  
  n_gender <- 2
  
  # explications de la fonction plus bas
  calculate_prob <- function(index_users,n,ratings_prob){
    newM <- m.sparse[unique(data_user[index_users,]$user.id),]
    r <- array(dim = c(n_item,5))
    for (i in 1:5){
      newM2 <- newM
      newM2[newM2!=i] <- 0
      r[,i] <-  colSums(newM2)/i
    }
    
    r <- (r + 1) / (ratings_prob + n)
    return(r)
  }
  
  # matrice 1682x5 contenant les probabilités P(profession|v=i) pour i=1...5 et pour chaque film
  prob_job_if_rating <- calculate_prob(index_job, n_job, array(n_ratings_by_film, dim=c(n_item,5)))
  
  # matrice 1682x5 contenant les probabilités P(age|v=i)pour i=1...5 et pour chaque film
  prob_age_if_rating <- calculate_prob(index_age, n_age, array(n_ratings_by_film, dim=c(n_item,5)))
  
  # matrice 1682x5 contenant les probabilités P(genre|v=i) pour i=1...5 et pour chaque film
  prob_gender_if_rating <- calculate_prob(index_gender, n_gender, array(n_ratings_by_film, dim=c(n_item,5)))
    
  # matrice 1682x5 contenant les probabilités P(v=i|profession,age,genre) pour i=1...5 et pour chaque film
  total_prob_by_rating <- prob_ratings_by_film * prob_job_if_rating * prob_age_if_rating * prob_gender_if_rating
    
  # vecteur de taille 1682 contenant l'estimation de vote pour chaque film à un facteur près
  expec_film <- total_prob_by_rating %*% 1:5
  
  prop <- max.nindex(expec_film,10)
  
  return(u.item[prop,'movie.title'])
}

La fonction *calculate_prob* calcule pour chaque film les probabilités $P(v=i|F) \ \ \ \forall i \in \{1...5\}$ où F est un facteur du type profession, âge ou genre. Elle renvoie une matrice de dimension (1682,5) où l'élément $(i,j)$ correspond à la probabilité $P(v=j|F)$ pour le film $i$.

*index_users* représente les indexs des utilisateurs qui répondent au facteur F (exemple : tous les utilisateurs ingénieurs, ou tout ceux de 30 ans, ...)

*n* est le nombre de type différents d'utilisateurs (2 pour les genres, 21 pour les professions, ...)

*ratings_prob* est la matrice de dimension (1682,5) où l'élément (i,j) correspond le nombre de votes de valeur j pour le film i

Voici les 10 films proposés par notre algorithme bayésien pour certains types d'utilisateurs.

In [ ]:
Bayes_recommendation('engineer',30,'M')
Bayes_recommendation('marketing',20,'F')
Bayes_recommendation('technician',60,'M')

### Q7 
*Reprenez les calculs de la question 5 avec une validation croisée. Utilisez cette technique pour déterminer le nombre de classes d'agglomération.*